# Integrating_Yolov8_with_VGG19

In [ ]:
!pip install tensorflow
!pip install ultralytics

In [ ]:
!pip install opencv-python
!pip install matplotlib

In [ ]:
# Load the trained model VGG19 
from tensorflow import keras

loaded_model = keras.models.load_model("../save/VGG19_Model.h5")

In [ ]:
#VGG91_to_predict_single_image

import cv2
import numpy as np

# Load  image

img = cv2.imread('../AI_High_Resolution/1.jpg')

#img = cv2.imread('../Real_High_Resolution/Diverse-group-of-people.jpg')



# Resize the image (assuming 224x224 input)
img = cv2.resize(img, (224, 224))

# Convert to NumPy array and normalize (assuming BGR format)
img = img.astype(np.float32) / 255.0

# Reshape the image to add a batch dimension
img_batch = np.expand_dims(img, axis=0)

#predictions 
predictions = loaded_model.predict(img_batch)

print(predictions)
print("The probability of the image belonging to the AI-images class : ",predictions[0][0])
print("The probability of the image belonging to the Real images class : ",predictions[0][1])

#print("lebel/class : ",predicted_class_index)
if predictions[0][0] > predictions[0][1]:
    print("Al-generated image")
else:
    print("Real image")

In [ ]:
# import Yolo
from ultralytics import YOLO

# Load a trained model yolov8

trained_model = YOLO("../save/YoloV8_Face_Detection_Model.pt")

In [ ]:
from PIL import Image
import cv2
#from google.colab.patches import cv2_imshow


source_image = cv2.imread('../AI_High_Resolution/1.jpg')

#source_image = cv2.imread('../Real_High_Resolution/Diverse-group-of-people.jpg')


results = trained_model(source_image , conf = 0.70)


In [ ]:
result = results[0]
from PIL import Image
Image.fromarray(result.plot()[:,:,::-1])

# CODE OF PREDICTION "CROPPING SQUARED BOUNDING BOX"

In [ ]:
from PIL import Image
from IPython.display import display
import numpy as np
import cv2

cropped_imgs = []

# Extract bounding boxes
boxes = results[0].boxes.xyxy.cpu().tolist()

# Iterate through each bounding box
for i, box in enumerate(boxes):
    
    x_min, y_min, x_max, y_max = map(int, box)  # Convert coordinates to integers for indexing

    # Calculate the width and height of the bounding box
    width = x_max - x_min
    height = y_max - y_min

    # Determine the size of the square
    side_length = max(width, height)

    # Calculate the center of the bounding box
    center_x = x_min + width // 2
    center_y = y_min + height // 2

    # Calculate the new square bounding box coordinates
    new_x_min = center_x - side_length // 2
    new_y_min = center_y - side_length // 2
    new_x_max = center_x + side_length // 2
    new_y_max = center_y + side_length // 2

    # Ensure the coordinates are within the image boundaries
    new_x_min = max(0, new_x_min)
    new_y_min = max(0, new_y_min)
    new_x_max = min(source_image.shape[1], new_x_max)
    new_y_max = min(source_image.shape[0], new_y_max)

    # Crop the image
    cropped_img = source_image[new_y_min:new_y_max, new_x_min:new_x_max]
    #print ( new_y_min, new_y_max, new_x_min , new_x_max)
    
    #######################
    #x1, y1, x2, y2 = map(int, box)  # Convert coordinates to integers for indexing
    #print ( x1, y1, x2, y2)
    # Crop the object using the bounding box coordinates
    #cropped_img = source_image[y1:y2, x1:x2]

    # Save the cropped image 
    cv2.imwrite(f'cropped_img{i}.jpg', cropped_img)
    
        
    # Resize the image (VGG19 --- 224x224 input)
    img = cv2.resize(cropped_img,(224,224))

    cropped_imgs.append(img)
        
print("No. of faces : ",len(cropped_imgs))        
# Convert to NumPy array and normalize 
cropped_imgs = np.array(cropped_imgs)

cropped_imgs = cropped_imgs/255.0

# Reshape the image to add a batch dimension
#img_batch = np.expand_dims(img, axis=0)
#print(img_batch.shape)


predictions = loaded_model.predict(cropped_imgs)

print("predictions probabilities:\n",predictions)

i = 0
for prediction in predictions:
    
    print('\n',prediction)
    print("The probability of the image belonging to the AI-images class : ",prediction[0])
    print("The probability of the image belonging to the Real images class : ",prediction[1])

    # Find the class with the highest probability
    # np.argmax returns the index/indices of the maximum value(s) along an axis. 
    # For example, if we have an array [1, 3, 2, 4], np.argmax would return 3 (index) because 4 is the highest value.
    # predicted_class_index = np.argmax(predictions[0])  # Access the first element (your image)

    
    if prediction[0] > prediction[1]:
        print("lebel/class : 0 ",)
        print("Al-generated image")
    else:
        print("lebel/class : 1 ")
        print("Real image")
    image_path = f'cropped_img{i}.jpg'
    i +=1
    try:
        # Open the image using PIL
        image = Image.open(image_path)
        display(image)
    except FileNotFoundError:
        print(f"Error: Image file '{image_path}' not found.")

